In [ ]:
# TO USE AFTER DENOISING IS FINALIZED 
# FIND TAXONOMY.QZA EQUIVALENT 


from qiime2 import Visualization
import os
import pandas as pd

# CHANGE THIS -- add denoised data
data = seqs-filtered.qza

# Find equivalent for 'taxonomy.qza'

# Data directory 
data_dir = 'phylo_data'
if not os.path.isdir(data_dir):
    os.makedirs(data_dir)

In [ ]:
# Multiple Alignment

! qiime alignment mafft \
    --i-sequences $data_dir/seqs-filtered.qza \
    --o-alignment $data_dir/aligned-seqs.qza

In [ ]:
# Alignment Masking (removing regions w/ ambiguous aligments)

! qiime alignment mask \
    --i-alignment $data_dir/aligned-seqs.qza \
    --o-masked-alignment $data_dir/masked-aligned-seqs.qza

In [ ]:
# DE NOVO Tree Construction

! qiime phylogeny fasttree \
    --i-alignment $data_dir/masked-aligned-seqs.qza \
    --o-tree $data_dir/fasttree.qza

! qiime phylogeny midpoint-root \
    --i-tree $data_dir/fasttree.qza \
    --o-rooted-tree $data_dir/fasttree-rooted.qza

In [ ]:
# DE NOVO Tree Visualisation 

! qiime empress tree-plot \
    --i-tree $data_dir/fasttree-rooted.qza \
    --m-feature-metadata-file $taxonomy.qza \
    --o-visualization $data_dir/fasttree-rooted.qzv

In [ ]:
Visualization.load(f'{data_dir}fasttree-rooted.qzv')

In [ ]:
# BOOTSRAPPING 

! qiime phylogeny raxml-rapid-bootstrap \
    --i-alignment $data_dir/masked-aligned-seqs.qza \
    --p-seed 1723 \
    --p-rapid-bootstrap-seed 9384 \
    --p-bootstrap-replicates 100 \
    --p-substitution-model GTRCAT \
    --p-n-threads 3 \
    --o-tree $data_dir/raxml-cat-bootstrap-tree.qza

In [ ]:
# DE NOVO - Visualize Bootstrapped Rooted Tree 
! qiime phylogeny midpoint-root \
    --i-tree $data_dir/raxml-cat-bootstrap-tree.qza \
    --o-rooted-tree $data_dir/raxml-cat-bootstrap-rooted.qza

! qiime empress tree-plot \
    --i-tree $data_dir/raxml-cat-bootstrap-rooted.qza \
    --m-feature-metadata-file $data_dir/taxonomy.qza \
    --o-visualization $data_dir/raxml-cat-bootstrap-rooted.qzv

In [ ]:
Visualization.load(f'{data_dir}/raxml-cat-bootstrap-rooted.qzv')

In [ ]:
# FRAGMENT INSERTION (Frag In) Reference Tree

! wget -nv -O $data_dir/sepp-refs-gg-13-8.qza https://data.qiime2.org/2021.4/common/sepp-refs-gg-13-8.qza

In [ ]:
# (slow step)
# FRAG IN Construct Tree

! qiime fragment-insertion sepp \
    --i-representative-sequences $data_dir/seqs-filtered.qza \
    --i-reference-database $data_dir/sepp-refs-gg-13-8.qza \
    --p-threads 2 \
    --o-tree $data_dir/sepp-tree.qza \
    --o-placements $data_dir/sepp-tree-placements.qza

In [1]:
# FRAG IN Visualize Tree
! qiime empress tree-plot \
    --i-tree $data_dir/sepp-tree.qza \
    --m-feature-metadata-file $data_dir/taxonomy.qza \
    --o-visualization $data_dir/sepp-tree.qzv

There was an issue with loading the file /taxonomy.qza as metadata:

  Metadata file path doesn't exist, or the path points to something other than a file. Please check that the path exists, has read permissions, and points to a regular file (not a directory): /taxonomy.qza

  There may be more errors present in the metadata file. To get a full report, sample/feature metadata files can be validated with Keemei: https://keemei.qiime2.org

  Find details on QIIME 2 metadata requirements here: https://docs.qiime2.org/2022.2/tutorials/metadata/



In [ ]:
Visualization.load(f'{data_dir}/sepp-tree.qzv')